# জেনারেটিভ নেটওয়ার্ক

Recurrent Neural Networks (RNNs) এবং তাদের গেটেড সেল ভ্যারিয়েন্ট যেমন Long Short Term Memory Cells (LSTMs) এবং Gated Recurrent Units (GRUs) ভাষা মডেলিংয়ের জন্য একটি পদ্ধতি প্রদান করে, অর্থাৎ তারা শব্দের ক্রম শিখতে পারে এবং একটি ক্রমে পরবর্তী শব্দের পূর্বাভাস দিতে পারে। এটি আমাদের RNNs ব্যবহার করে **জেনারেটিভ কাজ** সম্পাদন করতে সক্ষম করে, যেমন সাধারণ টেক্সট জেনারেশন, মেশিন ট্রান্সলেশন, এমনকি ইমেজ ক্যাপশনিং।

পূর্ববর্তী ইউনিটে আমরা যে RNN আর্কিটেকচার আলোচনা করেছি, সেখানে প্রতিটি RNN ইউনিট পরবর্তী হিডেন স্টেটকে আউটপুট হিসেবে তৈরি করত। তবে, আমরা প্রতিটি রিকারেন্ট ইউনিটে আরেকটি আউটপুট যোগ করতে পারি, যা আমাদের একটি **সিকোয়েন্স** আউটপুট করতে সাহায্য করবে (যা মূল সিকোয়েন্সের সমান দৈর্ঘ্যের হবে)। এছাড়াও, আমরা এমন RNN ইউনিট ব্যবহার করতে পারি যা প্রতিটি ধাপে ইনপুট গ্রহণ করে না, বরং শুধুমাত্র একটি প্রাথমিক স্টেট ভেক্টর গ্রহণ করে এবং তারপর একটি আউটপুট সিকোয়েন্স তৈরি করে।

এই নোটবুকে, আমরা সহজ জেনারেটিভ মডেলের উপর ফোকাস করব যা আমাদের টেক্সট জেনারেট করতে সাহায্য করে। সরলতার জন্য, চলুন **ক্যারেক্টার-লেভেল নেটওয়ার্ক** তৈরি করি, যা অক্ষর ধরে ধরে টেক্সট জেনারেট করে। প্রশিক্ষণের সময়, আমাদের কিছু টেক্সট কর্পাস নিতে হবে এবং এটিকে অক্ষরের সিকোয়েন্সে ভাগ করতে হবে।


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## চরিত্রের শব্দভাণ্ডার তৈরি করা

চরিত্র-স্তরের জেনারেটিভ নেটওয়ার্ক তৈরি করতে, আমাদের পাঠ্যকে শব্দের পরিবর্তে পৃথক অক্ষরে বিভক্ত করতে হবে। এটি একটি ভিন্ন টোকেনাইজার সংজ্ঞায়িত করে করা যেতে পারে:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


চলুন দেখি কীভাবে আমরা আমাদের ডেটাসেট থেকে টেক্সট এনকোড করতে পারি:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## একটি জেনারেটিভ RNN প্রশিক্ষণ করা

আমরা RNN-কে টেক্সট জেনারেট করার জন্য যেভাবে প্রশিক্ষণ দেব তা হলো নিম্নরূপ। প্রতিটি ধাপে, আমরা `nchars` দৈর্ঘ্যের একটি অক্ষরের ক্রম নেব এবং নেটওয়ার্ককে প্রতিটি ইনপুট অক্ষরের জন্য পরবর্তী আউটপুট অক্ষর তৈরি করতে বলব:

![চিত্রটি 'HELLO' শব্দের একটি উদাহরণ RNN জেনারেশন দেখাচ্ছে।](../../../../../translated_images/bn/rnn-generate.56c54afb52f9781d.webp)

বাস্তব পরিস্থিতির উপর নির্ভর করে, আমরা কিছু বিশেষ অক্ষর অন্তর্ভুক্ত করতে চাইতে পারি, যেমন *end-of-sequence* `<eos>`। আমাদের ক্ষেত্রে, আমরা শুধু নেটওয়ার্ককে অবিরাম টেক্সট জেনারেশনের জন্য প্রশিক্ষণ দিতে চাই, তাই আমরা প্রতিটি ক্রমের আকারকে `nchars` টোকেনের সমান স্থির করব। ফলস্বরূপ, প্রতিটি প্রশিক্ষণ উদাহরণে থাকবে `nchars` ইনপুট এবং `nchars` আউটপুট (যা ইনপুট ক্রম এক প্রতীক বামে সরানো)। মিনিব্যাচে এমন কয়েকটি ক্রম থাকবে।

আমরা যেভাবে মিনিব্যাচ তৈরি করব তা হলো প্রতিটি নিউজ টেক্সটের দৈর্ঘ্য `l` নিয়ে, এবং এর থেকে সমস্ত সম্ভাব্য ইনপুট-আউটপুট সংমিশ্রণ তৈরি করা (এখানে `l-nchars` এমন সংমিশ্রণ থাকবে)। এগুলো একটি মিনিব্যাচ গঠন করবে, এবং প্রতিটি প্রশিক্ষণ ধাপে মিনিব্যাচের আকার ভিন্ন হবে।


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

এখন আমরা জেনারেটর নেটওয়ার্ক সংজ্ঞায়িত করব। এটি পূর্ববর্তী ইউনিটে আলোচনা করা যেকোনো পুনরাবৃত্ত সেলের (সাধারণ, LSTM বা GRU) উপর ভিত্তি করে হতে পারে। আমাদের উদাহরণে আমরা LSTM ব্যবহার করব।

কারণ নেটওয়ার্কটি ইনপুট হিসাবে অক্ষর গ্রহণ করে এবং শব্দভাণ্ডারের আকার বেশ ছোট, তাই আমাদের এম্বেডিং লেয়ারের প্রয়োজন নেই; এক-হট-এনকোড করা ইনপুট সরাসরি LSTM সেলে যেতে পারে। তবে, যেহেতু আমরা ইনপুট হিসাবে অক্ষরের সংখ্যা পাস করি, তাই LSTM-এ পাঠানোর আগে সেগুলোকে এক-হট-এনকোড করতে হবে। এটি `forward` পাসের সময় `one_hot` ফাংশন কল করার মাধ্যমে সম্পন্ন হয়। আউটপুট এনকোডার হবে একটি লিনিয়ার লেয়ার, যা লুকানো অবস্থাকে এক-হট-এনকোড করা আউটপুটে রূপান্তর করবে।


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

প্রশিক্ষণের সময়, আমরা তৈরি করা টেক্সট থেকে নমুনা নিতে চাই। এটি করার জন্য, আমরা একটি `generate` ফাংশন সংজ্ঞায়িত করব যা প্রাথমিক স্ট্রিং `start` থেকে শুরু করে `size` দৈর্ঘ্যের একটি আউটপুট স্ট্রিং তৈরি করবে।

এর কাজ করার পদ্ধতি নিম্নরূপ। প্রথমে, আমরা পুরো `start` স্ট্রিংটি নেটওয়ার্কের মাধ্যমে পাঠাব এবং আউটপুট স্টেট `s` এবং পরবর্তী পূর্বাভাসিত অক্ষর `out` গ্রহণ করব। যেহেতু `out` এক-হট এনকোড করা থাকে, আমরা `argmax` ব্যবহার করে শব্দভাণ্ডারে অক্ষরের সূচক `nc` বের করব এবং `itos` ব্যবহার করে প্রকৃত অক্ষরটি নির্ধারণ করব এবং এটি `chars` নামক অক্ষরের তালিকায় যোগ করব। এই প্রক্রিয়াটি একবারে একটি অক্ষর তৈরি করার জন্য `size` বার পুনরাবৃত্তি করা হয়, যাতে প্রয়োজনীয় সংখ্যক অক্ষর তৈরি করা যায়।


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

এবার চলুন প্রশিক্ষণ শুরু করি! প্রশিক্ষণ লুপটি আমাদের আগের সব উদাহরণের মতোই, তবে এখানে নির্ভুলতার পরিবর্তে আমরা প্রতি ১০০০ এপোক পরপর নমুনা হিসেবে তৈরি করা টেক্সট প্রিন্ট করব।

বিশেষ মনোযোগ দিতে হবে যেভাবে আমরা লস গণনা করি। আমাদের লস গণনা করতে হবে এক-হট-এনকোড করা আউটপুট `out` এবং প্রত্যাশিত টেক্সট `text_out` (যা চরিত্রের ইনডেক্সের তালিকা) ব্যবহার করে। সৌভাগ্যক্রমে, `cross_entropy` ফাংশনটি প্রথম আর্গুমেন্ট হিসেবে অপরিশোধিত নেটওয়ার্ক আউটপুট এবং দ্বিতীয় আর্গুমেন্ট হিসেবে ক্লাস নম্বর আশা করে, যা আমাদের কাছে ঠিকই আছে। এটি মিনিব্যাচ সাইজের উপর স্বয়ংক্রিয় গড়ও করে।

আমরা প্রশিক্ষণকে `samples_to_train` নমুনা দিয়ে সীমাবদ্ধ রাখি, যাতে খুব বেশি সময় অপেক্ষা করতে না হয়। আমরা আপনাকে উৎসাহিত করি দীর্ঘ সময় ধরে প্রশিক্ষণ চালিয়ে দেখতে, সম্ভবত কয়েকটি এপোকের জন্য (যে ক্ষেত্রে আপনাকে এই কোডের চারপাশে আরেকটি লুপ তৈরি করতে হবে)।


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

এই উদাহরণটি ইতিমধ্যেই বেশ ভালো টেক্সট তৈরি করে, তবে এটি আরও কয়েকটি উপায়ে উন্নত করা যেতে পারে:

* **উন্নত মিনি-ব্যাচ জেনারেশন**। আমরা প্রশিক্ষণের জন্য ডেটা প্রস্তুত করার সময় এক নমুনা থেকে এক মিনি-ব্যাচ তৈরি করেছি। এটি আদর্শ নয়, কারণ মিনি-ব্যাচগুলোর আকার বিভিন্ন হয়, এবং কিছু ক্ষেত্রে মিনি-ব্যাচ তৈরি করা সম্ভব হয় না, কারণ টেক্সট `nchars` এর চেয়ে ছোট। এছাড়াও, ছোট মিনি-ব্যাচ GPU যথেষ্ট পরিমাণে লোড করতে পারে না। আরও বুদ্ধিমানের কাজ হবে সমস্ত নমুনা থেকে একটি বড় টেক্সটের অংশ নেওয়া, তারপর সমস্ত ইনপুট-আউটপুট জোড়া তৈরি করা, সেগুলোকে এলোমেলো করা এবং সমান আকারের মিনি-ব্যাচ তৈরি করা।

* **মাল্টিলেয়ার LSTM**। ২ বা ৩ স্তরের LSTM সেল ব্যবহার করার চেষ্টা করা যুক্তিসঙ্গত। আমরা আগের ইউনিটে উল্লেখ করেছি, LSTM এর প্রতিটি স্তর টেক্সট থেকে নির্দিষ্ট প্যাটার্ন বের করে, এবং ক্যারেক্টার-লেভেল জেনারেটরের ক্ষেত্রে আমরা আশা করতে পারি যে নিম্ন স্তরের LSTM শব্দাংশ বের করার জন্য দায়ী হবে, এবং উচ্চ স্তরগুলো শব্দ এবং শব্দের সংমিশ্রণ বের করবে। এটি সহজেই LSTM কনস্ট্রাক্টরে স্তরের সংখ্যা প্যারামিটার পাস করে বাস্তবায়ন করা যেতে পারে।

* আপনি **GRU ইউনিট** নিয়ে পরীক্ষা করতে পারেন এবং দেখতে পারেন কোনটি ভালো কাজ করে, এবং **বিভিন্ন হিডেন লেয়ার সাইজ** নিয়ে পরীক্ষা করতে পারেন। খুব বড় হিডেন লেয়ার ওভারফিটিং ঘটাতে পারে (যেমন, নেটওয়ার্ক সঠিক টেক্সট শিখে নেবে), এবং ছোট আকার ভালো ফলাফল দিতে পারে না।


## নরম টেক্সট জেনারেশন এবং টেম্পারেচার

`generate` এর পূর্ববর্তী সংজ্ঞায়, আমরা সর্বদা সর্বোচ্চ সম্ভাবনাযুক্ত চরিত্রটিকে পরবর্তী চরিত্র হিসেবে গ্রহণ করছিলাম। এর ফলে প্রায়ই টেক্সট একই চরিত্রের ক্রম বারবার পুনরাবৃত্তি করত, যেমন এই উদাহরণে:
```
today of the second the company and a second the company ...
```

তবে, যদি আমরা পরবর্তী চরিত্রের সম্ভাবনা বিতরণটি দেখি, তাহলে দেখা যেতে পারে যে কয়েকটি সর্বোচ্চ সম্ভাবনার মধ্যে পার্থক্য খুব বেশি নয়, যেমন একটি চরিত্রের সম্ভাবনা হতে পারে ০.২, আরেকটি - ০.১৯ ইত্যাদি। উদাহরণস্বরূপ, যদি আমরা '*play*' ক্রমের পরবর্তী চরিত্র খুঁজছি, তাহলে পরবর্তী চরিত্রটি সমানভাবে স্পেস হতে পারে, অথবা **e** (যেমন শব্দটি *player* এ)।

এটি আমাদের এই সিদ্ধান্তে পৌঁছায় যে সর্বোচ্চ সম্ভাবনাযুক্ত চরিত্রটি নির্বাচন করা সবসময় "ন্যায্য" নয়, কারণ দ্বিতীয় সর্বোচ্চটি নির্বাচন করলেও তা অর্থপূর্ণ টেক্সট তৈরি করতে পারে। এটি আরও বুদ্ধিমানের কাজ হবে যদি আমরা নেটওয়ার্ক আউটপুট দ্বারা প্রদত্ত সম্ভাবনা বিতরণ থেকে চরিত্রগুলি **নমুনা** করি।

এই নমুনা প্রক্রিয়া `multinomial` ফাংশন ব্যবহার করে করা যেতে পারে, যা তথাকথিত **multinomial distribution** বাস্তবায়ন করে। একটি ফাংশন যা এই **নরম** টেক্সট জেনারেশন বাস্তবায়ন করে তা নিচে সংজ্ঞায়িত করা হয়েছে:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

আমরা **temperature** নামে একটি অতিরিক্ত প্যারামিটার পরিচয় করিয়েছি, যা নির্দেশ করে যে আমাদের সর্বোচ্চ সম্ভাবনার প্রতি কতটা কঠোরভাবে অনুসরণ করা উচিত। যদি temperature 1.0 হয়, আমরা ন্যায্য মাল্টিনোমিয়াল স্যাম্পলিং করি, এবং যখন temperature অসীমে পৌঁছায় - সমস্ত সম্ভাবনা সমান হয়ে যায়, এবং আমরা পরবর্তী অক্ষরটি এলোমেলোভাবে নির্বাচন করি। নিচের উদাহরণে আমরা দেখতে পারি যে temperature খুব বেশি বাড়ালে টেক্সট অর্থহীন হয়ে যায়, এবং এটি "cycled" কঠোরভাবে তৈরি টেক্সটের মতো হয়ে যায় যখন এটি 0-এর কাছাকাছি হয়।



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
